In [ ]:
# install YOLOv8
#!pip install ultralytics
#!pip install roboflow

In [ ]:
# Cell 1 — Environment info
import sys, platform, torch
print("Python:", sys.version.split()[0])
print("Platform:", platform.platform())
print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA device count:", torch.cuda.device_count())
    for i in range(torch.cuda.device_count()):
        print(f"Device {i}:", torch.cuda.get_device_name(i))

else:
    print("No CUDA device detected. If you intended to use GPU, check CUDA / driver / torch installation. :contentReference[oaicite:4]{index=4}")


In [ ]:
# Cell 4 — Quick dataset check (optional visualization)
import glob, random, os
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt

def show_sample(img_dir, label_dir):
    imgs = glob.glob(os.path.join(img_dir, "*"))
    if not imgs:
        print("No images in", img_dir); return
    img_path = random.choice(imgs)
    base = os.path.splitext(os.path.basename(img_path))[0]
    label_path = os.path.join(label_dir, base + ".txt")
    img = Image.open(img_path).convert("RGB")
    w,h = img.size
    draw = ImageDraw.Draw(img)
    if os.path.exists(label_path):
        with open(label_path) as f:
            for line in f:
                cls, xc,yc,ww,hh = map(float, line.strip().split())
                xc,yc,ww,hh = xc*w, yc*h, ww*w, hh*h
                x0,y0, x1,y1 = xc-ww/2, yc-hh/2, xc+ww/2, yc+hh/2
                draw.rectangle([x0,y0,x1,y1], outline="red", width=2)
                draw.text((x0, y0), str(int(cls)), fill="yellow")
    plt.figure(figsize=(8,6)); plt.imshow(img); plt.axis("off"); plt.show()

# Example usage (edit paths if needed)
# show_sample("./dataset/images/train", "./dataset/labels/train")


In [ ]:
# training - adjust epochs as needed, and can reduce batch size if you get oom errors

from ultralytics import YOLO

model = YOLO("yolov8s.pt")

# Change as required
path = "C:/Users/ishaa/Desktop/AI_Testing_Ground/Model_Training/ECE4191/datasets/picam-noir-v1-061025/data.yaml"

# Train

model.train(
    data=path,                      # Path to your dataset YAML file
    imgsz=640,                      # Matches your dataset resize
    epochs=150,                     # Reasonable for medium dataset
    #batch=8,                       # Start with 8; increase to 16 if VRAM allows
    batch=-1,                       # Auto-finds max batch size before training
    workers=2,                      # Adjust based on CPU cores
    device=0,                       # Use GPU 0 (RTX 3060)
    half=True,                      # Mixed precision – saves ~30–40% VRAM
    optimizer='AdamW',              # Stable and efficient optimizer
    patience=30,                    # Early stopping if no improvement
    project='runs/train',           # Folder for training runs
    name='yolov8s_640_attempt2',   # Custom run name
    pretrained=True,                # Use pretrained weights
    cache=False,                     # Cache images for faster training
)


In [ ]:
# For resuming model if paused
#path_to_last = "runs/train/yolov8s_640_attempt2/weights/last.pt"
#model = YOLO(path_to_last)  # path to your last checkpoint
#model.train(resume=True)


In [ ]:
# Cell 6 — Evaluate model
metrics = model.val()  # runs validation
print(metrics)

In [ ]:
# Cell 8 — Export model for deployment
# Options: 'onnx', 'torchscript', 'tflite', 'engine' (TensorRT), etc.
model.export(format="onnx")


In [ ]:
#from ultralytics import YOLO

# Load a COCO-pretrained YOLOv8n model
#model = YOLO("yolov8n.pt")

# Display model information (optional)
#model.info()

# Train the model on the COCO8 example dataset for 100 epochs
#results = model.train(data="coco8.yaml", epochs=100, imgsz=640)

# Run inference with the YOLOv8n model on the 'bus.jpg' image
#results = model("path/to/bus.jpg")